# Tokenizers

In NLP, tokenizers translate text into data ready to be processed by the model. There are different approaches, such as word, character and subword level, BPE, SentencePiece, etc. Each pre-trained model is coupled with the tokenizer that was used to process its training data. As we did with models, tokenizers are loaded with the name of the checkpoint

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-cased")